# Experiment 01

## Core

In [1]:
import numpy, os, pickle, time
from matplotlib import pyplot
from utils import SimulationData, build_banded_matrix
%matplotlib inline

In [2]:
# Initialize data
data = SimulationData(
    R=3e-3, L=1e-3, G=1e-2, C=1e3,
    X=1.0, T=1.0, num_x_points=100, num_t_points=1000,
    mu_0 = lambda x : numpy.sin(5*numpy.pi*x) + 2*numpy.sin(7*numpy.pi*x),
    xi_0 = 0.,
    nu_0 = lambda t : 0., nu_X = lambda t : 0.,
    name="experiment_01"
)
print(f"CFL number = {data.get_CFL_number()}\n")

Line parameters initialized.
Domain parameters initialized.
CFL number = 0.09909909909909909



In [3]:
# Experiment parameters
num_x_points = [int(100*i) for i in [1, 1.5, 2, 2.5, 3]]
num_t_points = [int(1000*i) for i in [1, 1.5, 2, 2.5, 3]]
num_runs = 20
runtimes = numpy.zeros((len(num_x_points),num_runs))
save_dir = "./"

## Numerical solution: basic update scheme

In [4]:
# Run experiment
for i, (K_plus_1, N_plus_1) in enumerate(zip(num_x_points, num_t_points)):
    print(f">> Simulating with K = {K_plus_1-1} and N = {N_plus_1-1}")
    # Set data
    data.set_domain(num_x_points=K_plus_1, num_t_points=N_plus_1)
    print(f"  CFL number = {data.get_CFL_number()}")

    # Update scheme parameters
    A = 1. + 0.5 * data.domain["t step"] * (data.line["alpha"] + data.line["beta"])
    B = 1. - 0.5 * data.domain["t step"] * (data.line["alpha"] + data.line["beta"])
    E = data.line["c squared"] * data.domain["t step"]**2 / data.domain["x step"]**2
    F = 2. - (2. * data.line["c squared"] * data.domain["t step"]**2 / data.domain["x step"]**2) - (data.line["alpha"] * data.line["beta"] * data.domain["t step"]**2)

    # Get discretized domains
    x_domain = data.get_x_domain()
    t_domain = data.get_t_domain()
    
    for j in range(num_runs):
        print(f"Run no. {j+1}...", end="")
        
        # Initialize container for voltages
        u = numpy.zeros((data.domain["K"]+1, data.domain["N"]+1))
        
        # Start counter
        tic = time.perf_counter()
        
        # Initial conditions
        u[:,0] = data.init_condition_value(x_domain)
        u[0,0] = data.dirichlet_sending_end(t_domain[0])
        u[-1,0] = data.dirichlet_receiving_end(t_domain[0])
        u[:,1] = data.init_condition_deriv(u[:,0])
        u[0,1] = data.dirichlet_sending_end(t_domain[1])
        u[-1,1] = data.dirichlet_receiving_end(t_domain[1])

        # n=1,2,...,N-1
        for n in range(1,data.domain["N"]):
            # k=1,2,...,K-1
            for k in range(1,data.domain["K"]):
                u_was, u_now = u[k,n-1], u[k,n]
                u_now_left, u_now_right = u[k-1,n], u[k+1,n]
                u_nxt = E*u_now_left + F*u_now + E*u_now_right - B*u_was
                u_nxt /= A
                u[k,n+1] = u_nxt
            # k=0, K
            u[0,n+1] = data.dirichlet_sending_end(t_domain[n+1])
            u[-1,n+1] = data.dirichlet_receiving_end(t_domain[n+1])
        
        # Stop counter
        toc = time.perf_counter()
        
        # Record elapsed time
        runtimes[i,j] = toc-tic
        print(f"completed in {toc-tic} seconds.")

del A, B, E, F
del x_domain, t_domain
del i, K_plus_1, N_plus_1
del j, u, tic, toc
del n, k, u_was, u_now, u_now_left, u_now_right, u_nxt

>> Simulating with K = 99 and N = 999
Domain parameters 'K' and 'x step' are adjusted.
Domain parameters 'N' and 't step' are adjusted.
  CFL number = 0.09909909909909909
Run no. 1...completed in 0.20767310000000005 seconds.
Run no. 2...completed in 0.19349029999999967 seconds.
Run no. 3...completed in 0.1891058999999995 seconds.
Run no. 4...completed in 0.19241229999999998 seconds.
Run no. 5...completed in 0.21363339999999997 seconds.
Run no. 6...completed in 0.20108939999999986 seconds.
Run no. 7...completed in 0.2077928 seconds.
Run no. 8...completed in 0.19214800000000043 seconds.
Run no. 9...completed in 0.19944760000000006 seconds.
Run no. 10...completed in 0.19800750000000011 seconds.
Run no. 11...completed in 0.19644209999999962 seconds.
Run no. 12...completed in 0.19631929999999986 seconds.
Run no. 13...completed in 0.20176879999999997 seconds.
Run no. 14...completed in 0.19967239999999986 seconds.
Run no. 15...completed in 0.19435530000000067 seconds.
Run no. 16...completed i

In [5]:
# Save results
with open(os.path.join(save_dir, "basic-update-scheme.pkl"), 'wb') as file:
        pickle.dump(runtimes, file)
print("Runtime results saved to " + os.path.join(save_dir, "basic-update-scheme.pkl") + ".")
del file

Runtime results saved to ./basic-update-scheme.pkl.


## Numerical solution: vectorized update scheme

In [4]:
# Run experiment
for i, (K_plus_1, N_plus_1) in enumerate(zip(num_x_points, num_t_points)):
    print(f">> Simulating with K = {K_plus_1-1} and N = {N_plus_1-1}")
    # Set data
    data.set_domain(num_x_points=K_plus_1, num_t_points=N_plus_1)
    print(f"  CFL number = {data.get_CFL_number()}")

    # Update scheme parameters
    A = 1. + 0.5 * data.domain["t step"] * (data.line["alpha"] + data.line["beta"])
    B = 1. - 0.5 * data.domain["t step"] * (data.line["alpha"] + data.line["beta"])
    E = data.line["c squared"] * data.domain["t step"]**2 / data.domain["x step"]**2
    F = 2. - (2. * data.line["c squared"] * data.domain["t step"]**2 / data.domain["x step"]**2) - (data.line["alpha"] * data.line["beta"] * data.domain["t step"]**2)
    E_matrix = build_banded_matrix(numpy.array([E, F, E]), data.domain["K"]-1)
    B_matrix = B*numpy.concatenate(
        (numpy.zeros((data.domain["K"]-1,1)), numpy.identity(data.domain["K"]-1), numpy.zeros((data.domain["K"]-1,1))),
        axis=1
    )

    # Get discretized domains
    x_domain = data.get_x_domain()
    t_domain = data.get_t_domain()
    
    for j in range(num_runs):
        print(f"Run no. {j+1}...", end="")
        
        # Initialize container for voltages
        u = numpy.zeros((data.domain["K"]+1, data.domain["N"]+1))
        
        # Start counter
        tic = time.perf_counter()
        
        # Initial conditions
        u[:,0] = data.init_condition_value(x_domain)
        u[0,0] = data.dirichlet_sending_end(t_domain[0])
        u[-1,0] = data.dirichlet_receiving_end(t_domain[0])
        u[:,1] = data.init_condition_deriv(u[:,0])
        u[0,1] = data.dirichlet_sending_end(t_domain[1])
        u[-1,1] = data.dirichlet_receiving_end(t_domain[1])

        # n=1,2,...,N-1
        for n in range(1,data.domain["N"]):
            u_was, u_now = u[:,n-1], u[:,n]
            # k=1,2,...,K-1
            hat_u_nxt = numpy.dot(E_matrix, u_now) - numpy.dot(B_matrix, u_was)
            hat_u_nxt /= A
            # k=0,1,...,K
            u_nxt = numpy.concatenate(
                (numpy.array([data.dirichlet_sending_end(t_domain[n+1])]),
                 hat_u_nxt,
                 numpy.array([data.dirichlet_receiving_end(t_domain[n+1])]))
            )
            u[:,n+1] = u_nxt
        
        # Stop counter
        toc = time.perf_counter()
        
        # Record elapsed time
        runtimes[i,j] = toc-tic
        print(f"completed in {toc-tic} seconds.")

del A, B, E, F, E_matrix, B_matrix
del x_domain, t_domain
del i, K_plus_1, N_plus_1
del j, u, tic, toc
del n, u_was, u_now, hat_u_nxt, u_nxt

>> Simulating with K = 99 and N = 999
Domain parameters 'K' and 'x step' are adjusted.
Domain parameters 'N' and 't step' are adjusted.
  CFL number = 0.09909909909909909
Run no. 1...completed in 0.1059126999999993 seconds.
Run no. 2...completed in 0.09514620000000029 seconds.
Run no. 3...completed in 0.09631420000000013 seconds.
Run no. 4...completed in 0.09497089999999986 seconds.
Run no. 5...completed in 0.09650910000000046 seconds.
Run no. 6...completed in 0.09693170000000073 seconds.
Run no. 7...completed in 0.09905770000000125 seconds.
Run no. 8...completed in 0.09845939999999942 seconds.
Run no. 9...completed in 0.09717120000000001 seconds.
Run no. 10...completed in 0.09664810000000124 seconds.
Run no. 11...completed in 0.09717080000000067 seconds.
Run no. 12...completed in 0.09626360000000034 seconds.
Run no. 13...completed in 0.09815190000000129 seconds.
Run no. 14...completed in 0.09751239999999939 seconds.
Run no. 15...completed in 0.09728660000000033 seconds.
Run no. 16...c

In [5]:
# Save results
with open(os.path.join(save_dir, "vectorized-update-scheme.pkl"), 'wb') as file:
        pickle.dump(runtimes, file)
print("Runtime results saved to " + os.path.join(save_dir, "vectorized-update-scheme.pkl") + ".")
del file

Runtime results saved to ./vectorized-update-scheme.pkl.


## Results summary